## Aneks preprocessing

In [7]:
import pandas as pd

aneks = pd.read_csv('../data/aneks/raw/anekdotov_net.csv')
aneks2 = pd.read_csv('../data/aneks/raw/anekdotov_net2.csv')
rofls = pd.read_csv('../data/aneks/raw/anekdotov_net_microrofls.csv')
total = pd.concat([aneks, aneks2, rofls], axis=0)

total = total.drop_duplicates('anek')
total = total['anek']

total.to_csv('../data/aneks/processed/aneks.csv', sep=';')

## Vocab preprocessing

In [6]:
import json

tokens = {}
embeddings = {}
with open('../data/vocab/raw/bpe.txt', 'r', encoding='utf-8') as f:
    for idx, line in enumerate(f):
        token, values = line.split(' ', maxsplit=1)
        tokens[token] = idx
        embeddings[idx] = list(map(float,values.split()))

with open('../data/vocab/processed/word2idx.json', 'w') as fp:
    json.dump(tokens, fp)

with open('../data/vocab/processed/idx2embedding.json', 'w') as fp:
    json.dump(embeddings, fp)